In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import KFold
from sklearn import preprocessing
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn import metrics
from mlxtend.feature_selection import ColumnSelector
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import collections
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [21]:
path = '/Users/charlie/Desktop/insurance/'

In [22]:
tr = pd.read_csv(path+'tr9.csv')
te = pd.read_csv(path+'te9.csv')
#policy = pd.read_csv(path+'policy2.csv')
#claim = pd.read_csv(path+'claim_0702.csv')
#df = pd.read_csv(path+'abt.csv')

# Merge ABT dataset

In [23]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
tr["qpt_qp_v"] = le.fit_transform(tr.qpt_gp)
te["qpt_qp_v"] = le.transform(te.qpt_gp)

In [25]:
tr = tr.drop(["qpt_gp","note"],1)
te = te.drop(["qpt_gp","note"],1)

In [8]:
df_features = [col for col in df.columns.values]
tr_features = [col for col in tr.columns.values]

In [15]:
tmp = df[["Policy_Number",'IC_04M_amt',
 'IC_04M_ddctb',
 'IC_04M_premm',
 'IC_05E_amt1',
 'IC_05E_amt3',
 'IC_05E_premm',
 'IC_05N_amt',
 'IC_05N_ddctb',
 'IC_05N_premm',
 'IC_09A_amt',
 'IC_09A_premm',
 'IC_12L_amt1',
 'IC_12L_amt3',
 'IC_12L_premm',
 'IC_12L_times',
 'IC_16G_Times',
 'IC_16G_amt1',
 'IC_16G_amt3',
 'IC_16G_premm',
 'IC_16P_amt',
 'IC_16P_premm',
 'IC_18A_amt1',
 'IC_18A_amt3',
 'IC_18A_premm',
 'IC_18A_times',
 'IC_29B_amt',
 'IC_29B_premm',
 'IC_29K_amt',
 'IC_29K_premm',
         'MPwT',
          'is_claim_by_car',
 'is_claim_by_person',
          'note',
          'paid_tot',
          "qpt_gp",
          
         ]]

In [17]:
tr = pd.merge(tr, tmp, on="Policy_Number", how='left')
te = pd.merge(te, tmp, on="Policy_Number", how='left')

# Additional features

In [79]:
tr["tot_premium_pdmg"] = tr.tot_premium * tr.pdmg_acc
te["tot_premium_pdmg"] = te.tot_premium * te.pdmg_acc

In [84]:
tr["tot_premium_pdmg_lia"] = tr.tot_premium * tr.pdmg_acc * tr.lia_class
te["tot_premium_pdmg_lia"] = te.tot_premium * te.pdmg_acc * te.lia_class

tr["tot_premium_pdmg_plia"] = tr.tot_premium * tr.pdmg_acc * tr.plia_acc
te["tot_premium_pdmg_plia"] = te.tot_premium * te.pdmg_acc * te.plia_acc

tr["tot_premium_lia_plia"] = tr.tot_premium * tr.lia_class * tr.plia_acc
te["tot_premium_lia_plia"] = te.tot_premium * te.lia_class * te.plia_acc

tr["tot_premium_lia_plia_pdmg"] = tr.tot_premium * tr.lia_class * tr.plia_acc * tr.pdmg_acc
te["tot_premium_lia_plia_pdmg"] = te.tot_premium * te.lia_class * te.plia_acc * te.pdmg_acc

In [92]:
tr["fsex_tot_premium"] = tr.fsex * tr.tot_premium
te["fsex_tot_premium"] = te.fsex * te.tot_premium

In [93]:
tr["tot_premium_engine"] = tr.tot_premium * tr["Engine_Displacement_(Cubic_Centimeter)"]
te["tot_premium_engine"] = te.tot_premium * te["Engine_Displacement_(Cubic_Centimeter)"]

In [94]:
tr["Claim_Number_tot_premium"] = tr.tot_premium * tr["Claim_Number"]
te["Claim_Number_tot_premium"] = te.tot_premium * te["Claim_Number"]

In [95]:
tr["claimants_tot_premium"] = tr.tot_premium * tr.sum_claimants
te["claimants_tot_premium"] = te.tot_premium * te.sum_claimants

In [96]:
tr["tot_premium_div_age"] = tr.tot_premium / tr.age
te["tot_premium_div_age"] = te.tot_premium / te.age

In [98]:
tr["fsex_tot_premium_age"] = tr.fsex * tr.tot_premium * tr.age
te["fsex_tot_premium_age"] = te.fsex * te.tot_premium * te.age

In [99]:
tr["tot_premium_engine_pdmg"] = tr.tot_premium * tr["Engine_Displacement_(Cubic_Centimeter)"] * tr.pdmg_acc
te["tot_premium_engine_pdmg"] = te.tot_premium * te["Engine_Displacement_(Cubic_Centimeter)"] * te.pdmg_acc

In [100]:
tr["tot_premium_engine_lia"] = tr.tot_premium * tr["Engine_Displacement_(Cubic_Centimeter)"] * tr.lia_class
te["tot_premium_engine_lia"] = te.tot_premium * te["Engine_Displacement_(Cubic_Centimeter)"] * te.lia_class

In [101]:
tr["tot_premium_engine_plia"] = tr.tot_premium * tr["Engine_Displacement_(Cubic_Centimeter)"] * tr.plia_acc
te["tot_premium_engine_plia"] = te.tot_premium * te["Engine_Displacement_(Cubic_Centimeter)"] * te.plia_acc

In [7]:
tr["A_type"] = tr.Replacement_cost_of_insured_vehicle * 0.1
te["A_type"] = te.Replacement_cost_of_insured_vehicle * 0.1

tr["B_type"] = tr.Replacement_cost_of_insured_vehicle * 0.05
te["B_type"] = te.Replacement_cost_of_insured_vehicle * 0.05

tr["C_type"] = tr.Replacement_cost_of_insured_vehicle * 0.02
te["C_type"] = te.Replacement_cost_of_insured_vehicle * 0.02

In [21]:
tmp = pd.concat([tr, te], axis=0)
tmp = tmp.groupby("Insured_ID", as_index= 0)["Claim_Number"].sum()
tmp.columns = ["Insured_ID", "Insured_ID_Claim_Number"]
tr = pd.merge(tr, tmp, on="Insured_ID", how='left')
te = pd.merge(te, tmp, on="Insured_ID", how='left')

In [ ]:
tmp = pd.concat([tr, te], axis=0)
tmp = tmp.groupby("Insured_ID", as_index= 0)["Claim_Number"].sum()
tmp.columns = ["Insured_ID", "Insured_ID_Claim_Number"]
tr = pd.merge(tr, tmp, on="Insured_ID", how='left')
te = pd.merge(te, tmp, on="Insured_ID", how='left')

In [32]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
policy["identifier"] = le.fit_transform(policy.Vehicle_identifier)

In [35]:
tmp = policy.groupby("Policy_Number", as_index=0)["identifier"].first()
tr = pd.merge(tr, tmp, on="Policy_Number", how='left')
te = pd.merge(te, tmp, on="Policy_Number", how='left')

In [39]:
df = pd.concat([tr, te], axis=0)
tmp = df.groupby("identifier", as_index= 0)["Claim_Number"].sum()

In [43]:
tmp.columns = ["identifier", "identifier_Claim_Number"]
tr = pd.merge(tr, tmp, on="identifier", how='left')
te = pd.merge(te, tmp, on="identifier", how='left')

In [52]:
tr["car_age_multiply_age"] = tr.car_age * tr.age
te["car_age_multiply_age"] = te.car_age * te.age

In [61]:
tmp = pd.concat([tr, te], axis=0)
tmp = tmp.groupby("Insured_ID", as_index= 0)["no_fault_count"].sum()
tmp.columns = ["Insured_ID", "Insured_ID_no_fault_count"]
tr = pd.merge(tr, tmp, on="Insured_ID", how='left')
te = pd.merge(te, tmp, on="Insured_ID", how='left')

In [64]:
tmp = pd.concat([tr, te], axis=0)
tmp = tmp.groupby("identifier", as_index= 0)["no_fault_count"].sum()
tmp.columns = ["identifier", "identifier_no_fault_count"]
tr = pd.merge(tr, tmp, on="identifier", how='left')
te = pd.merge(te, tmp, on="identifier", how='left')

In [68]:
tmp = policy.groupby("identifier", as_index=0).agg({"Policy_Number":'nunique'})
tmp.columns = ["identifier", "identifier_policy_Number"]
tr = pd.merge(tr, tmp, on="identifier", how='left')
te = pd.merge(te, tmp, on="identifier", how='left')

In [70]:
tr["repl_claim_num"] = tr.Replacement_cost_of_insured_vehicle * tr.Claim_Number
te["repl_claim_num"] = te.Replacement_cost_of_insured_vehicle * te.Claim_Number

In [15]:
tmp = policy[policy.Premium == 0]
tmp = tmp.groupby("Policy_Number", as_index=0)["Premium"].count()
tmp.columns = ["Policy_Number", "num_zero_premium"]
tr = pd.merge(tr, tmp, on="Policy_Number", how='left')
te = pd.merge(te, tmp, on="Policy_Number", how='left')

In [53]:
# 被保險人年齡分組
tr.loc[tr.age < 20, 'age_class'] = 0
tr.loc[(tr.age >= 20)&( tr.age < 25), 'age_class'] = 1
tr.loc[(tr.age >= 25)&( tr.age < 30), 'age_class'] = 2
tr.loc[(tr.age >= 30)&( tr.age < 60), 'age_class'] = 3
tr.loc[(tr.age >= 60)&( tr.age < 70), 'age_class'] = 4
tr.loc[tr.age >= 70, 'age_class'] = 5

te.loc[te.age < 20, 'age_class'] = 0
te.loc[(te.age >= 20)&( te.age < 25), 'age_class'] = 1
te.loc[(te.age >= 25)&( te.age < 30), 'age_class'] = 2
te.loc[(te.age >= 30)&( te.age < 60), 'age_class'] = 3
te.loc[(te.age >= 60)&( te.age < 70), 'age_class'] = 4
te.loc[te.age >= 70, 'age_class'] = 5

In [5]:
#fsex = 1 男 、 2 女
# age_class
tr.loc[(tr.fsex == 1) & (tr.age_class == 0), 'third_human_coefficient'] = 1.89
tr.loc[(tr.fsex == 1) & (tr.age_class == 1), 'third_human_coefficient'] = 1.74
tr.loc[(tr.fsex == 1) & (tr.age_class == 2), 'third_human_coefficient'] = 1.15
tr.loc[(tr.fsex == 1) & (tr.age_class == 3), 'third_human_coefficient'] = 1
tr.loc[(tr.fsex == 1) & (tr.age_class == 4), 'third_human_coefficient'] = 1.07
tr.loc[(tr.fsex == 1) & (tr.age_class == 5), 'third_human_coefficient'] = 1.07

te.loc[(te.fsex == 1) & (te.age_class == 0), 'third_human_coefficient'] = 1.89
te.loc[(te.fsex == 1) & (te.age_class == 1), 'third_human_coefficient'] = 1.74
te.loc[(te.fsex == 1) & (te.age_class == 2), 'third_human_coefficient'] = 1.15
te.loc[(te.fsex == 1) & (te.age_class == 3), 'third_human_coefficient'] = 1
te.loc[(te.fsex == 1) & (te.age_class == 4), 'third_human_coefficient'] = 1.07
te.loc[(te.fsex == 1) & (te.age_class == 5), 'third_human_coefficient'] = 1.07

tr.loc[(tr.fsex == 2) & (tr.age_class == 0), 'third_human_coefficient'] = 1.7
tr.loc[(tr.fsex == 2) & (tr.age_class == 1), 'third_human_coefficient'] = 1.57
tr.loc[(tr.fsex == 2) & (tr.age_class == 2), 'third_human_coefficient'] = 1.04
tr.loc[(tr.fsex == 2) & (tr.age_class == 3), 'third_human_coefficient'] = 0.9
tr.loc[(tr.fsex == 2) & (tr.age_class == 4), 'third_human_coefficient'] = 0.96
tr.loc[(tr.fsex == 2) & (tr.age_class == 5), 'third_human_coefficient'] = 0.96

te.loc[(te.fsex == 2) & (te.age_class == 0), 'third_human_coefficient'] = 1.7
te.loc[(te.fsex == 2) & (te.age_class == 1), 'third_human_coefficient'] = 1.57
te.loc[(te.fsex == 2) & (te.age_class == 2), 'third_human_coefficient'] = 1.04
te.loc[(te.fsex == 2) & (te.age_class == 3), 'third_human_coefficient'] = 0.9
te.loc[(te.fsex == 2) & (te.age_class == 4), 'third_human_coefficient'] = 0.96
te.loc[(te.fsex == 2) & (te.age_class == 5), 'third_human_coefficient'] = 0.96

In [54]:
#fsex = 1 男 、 2 女
# age_class
tr.loc[(tr.fsex == 1) & (tr.age_class == 0), 'human_coefficient'] = 2.5
tr.loc[(tr.fsex == 1) & (tr.age_class == 1), 'human_coefficient'] = 2.3
tr.loc[(tr.fsex == 1) & (tr.age_class == 2), 'human_coefficient'] = 1.47
tr.loc[(tr.fsex == 1) & (tr.age_class == 3), 'human_coefficient'] = 1
tr.loc[(tr.fsex == 1) & (tr.age_class >= 4), 'human_coefficient'] = 1.05

te.loc[(te.fsex == 1) & (te.age_class == 0), 'human_coefficient'] = 2.5
te.loc[(te.fsex == 1) & (te.age_class == 1), 'human_coefficient'] = 2.3
te.loc[(te.fsex == 1) & (te.age_class == 2), 'human_coefficient'] = 1.47
te.loc[(te.fsex == 1) & (te.age_class == 3), 'human_coefficient'] = 1
te.loc[(te.fsex == 1) & (te.age_class >= 4), 'human_coefficient'] = 1.05


tr.loc[(tr.fsex == 2) & (tr.age_class == 0), 'human_coefficient'] = 1.66
tr.loc[(tr.fsex == 2) & (tr.age_class == 1), 'human_coefficient'] = 1.53
tr.loc[(tr.fsex == 2) & (tr.age_class == 2), 'human_coefficient'] = 1.06
tr.loc[(tr.fsex == 2) & (tr.age_class == 3), 'human_coefficient'] = 0.92
tr.loc[(tr.fsex == 2) & (tr.age_class >= 4), 'human_coefficient'] = 0.79

te.loc[(te.fsex == 2) & (te.age_class == 0), 'human_coefficient'] = 1.66
te.loc[(te.fsex == 2) & (te.age_class == 1), 'human_coefficient'] = 1.53
te.loc[(te.fsex == 2) & (te.age_class == 2), 'human_coefficient'] = 1.06
te.loc[(te.fsex == 2) & (te.age_class == 3), 'human_coefficient'] = 0.92
te.loc[(te.fsex == 2) & (te.age_class >= 4), 'human_coefficient'] = 0.79

In [66]:
# 被保險人年齡分組
tr.loc[tr["Engine_Displacement_(Cubic_Centimeter)"] <= 500, 'engine_class'] = 1
tr.loc[(tr["Engine_Displacement_(Cubic_Centimeter)"] >= 501)&( tr["Engine_Displacement_(Cubic_Centimeter)"] <= 600), 'engine_class'] = 2
tr.loc[(tr["Engine_Displacement_(Cubic_Centimeter)"] >= 601)&( tr["Engine_Displacement_(Cubic_Centimeter)"] <= 1200), 'engine_class'] = 3
tr.loc[(tr["Engine_Displacement_(Cubic_Centimeter)"] >= 1201)&( tr["Engine_Displacement_(Cubic_Centimeter)"] <= 1800), 'engine_class'] = 4
tr.loc[(tr["Engine_Displacement_(Cubic_Centimeter)"] >= 1801)&( tr["Engine_Displacement_(Cubic_Centimeter)"] <= 2400), 'engine_class'] = 5
tr.loc[(tr["Engine_Displacement_(Cubic_Centimeter)"] >= 2401)&( tr["Engine_Displacement_(Cubic_Centimeter)"] <= 3000), 'engine_class'] = 6
tr.loc[(tr["Engine_Displacement_(Cubic_Centimeter)"] >= 3001)&( tr["Engine_Displacement_(Cubic_Centimeter)"] <= 4200), 'engine_class'] = 7 
tr.loc[(tr["Engine_Displacement_(Cubic_Centimeter)"] >= 4201)&( tr["Engine_Displacement_(Cubic_Centimeter)"] <= 5400), 'engine_class'] = 8
tr.loc[(tr["Engine_Displacement_(Cubic_Centimeter)"] >= 5401)&( tr["Engine_Displacement_(Cubic_Centimeter)"] <= 6600), 'engine_class'] = 9
tr.loc[(tr["Engine_Displacement_(Cubic_Centimeter)"] >= 6601)&( tr["Engine_Displacement_(Cubic_Centimeter)"] <= 7800), 'engine_class'] = 10
tr.loc[tr["Engine_Displacement_(Cubic_Centimeter)"] > 7800, 'engine_class'] = 11   



te.loc[te["Engine_Displacement_(Cubic_Centimeter)"] <= 500, 'engine_class'] = 1
te.loc[(te["Engine_Displacement_(Cubic_Centimeter)"] >= 501)&( te["Engine_Displacement_(Cubic_Centimeter)"] <= 600), 'engine_class'] = 2
te.loc[(te["Engine_Displacement_(Cubic_Centimeter)"] >= 601)&( te["Engine_Displacement_(Cubic_Centimeter)"] <= 1200), 'engine_class'] = 3
te.loc[(te["Engine_Displacement_(Cubic_Centimeter)"] >= 1201)&( te["Engine_Displacement_(Cubic_Centimeter)"] <= 1800), 'engine_class'] = 4
te.loc[(te["Engine_Displacement_(Cubic_Centimeter)"] >= 1801)&( te["Engine_Displacement_(Cubic_Centimeter)"] <= 2400), 'engine_class'] = 5
te.loc[(te["Engine_Displacement_(Cubic_Centimeter)"] >= 2401)&( te["Engine_Displacement_(Cubic_Centimeter)"] <= 3000), 'engine_class'] = 6
te.loc[(te["Engine_Displacement_(Cubic_Centimeter)"] >= 3001)&( te["Engine_Displacement_(Cubic_Centimeter)"] <= 4200), 'engine_class'] = 7 
te.loc[(te["Engine_Displacement_(Cubic_Centimeter)"] >= 4201)&( te["Engine_Displacement_(Cubic_Centimeter)"] <= 5400), 'engine_class'] = 8
te.loc[(te["Engine_Displacement_(Cubic_Centimeter)"] >= 5401)&( te["Engine_Displacement_(Cubic_Centimeter)"] <= 6600), 'engine_class'] = 9
te.loc[(te["Engine_Displacement_(Cubic_Centimeter)"] >= 6601)&( te["Engine_Displacement_(Cubic_Centimeter)"] <= 7800), 'engine_class'] = 10
te.loc[te["Engine_Displacement_(Cubic_Centimeter)"] > 7800, 'engine_class'] = 11


In [81]:
tmp = policy.groupby("Policy_Number", as_index=0)[["Vehicle_Make_and_Model1","Engine_Displacement_(Cubic_Centimeter)","qpt","Coding_of_Vehicle_Branding_&_Type",
                                                   "Replacement_cost_of_insured_vehicle","fequipment1",
                                            "fequipment2","fequipment3","fequipment4","fequipment5","fequipment6",
                                            "fequipment9","nequipment9"]].first()

In [37]:
# 0:輕型摩托車 1
tmp.loc[(tmp.qpt <= 2) & (tmp["Engine_Displacement_(Cubic_Centimeter)"] <=50),"vehicle_type"] = 0

# 2: 大型重型機車 17676
tmp.loc[(tmp.qpt <= 2) & (tmp["Engine_Displacement_(Cubic_Centimeter)"] >250),"vehicle_type"] = 2

,Policy_Number,Vehicle_Make_and_Model1,Engine_Displacement_(Cubic_Centimeter),qpt,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9
33785,187a117d958dd781781f43ea623633bd6d249ed3,7f92312f8d18f6c9b75704c5bb9a9b8f500b47c5,10,2.0,0,0,0,0,0,0,0,


In [128]:
policy["nequipment9_v"] = policy.nequipment9.str.replace(" ","")

In [129]:
policy.nequipment9_v.value_counts()

          1747906
大包             11
5合1影音          10
伸尾              9
原裝車含配備          6
Name: nequipment9_v, dtype: int64

In [131]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
policy["nequipment9_v2"] = le.fit_transform(policy.nequipment9_v)

In [135]:
tmp = policy.groupby("Policy_Number", as_index=0)["nequipment9_v2"].first()
tr = pd.merge(tr, tmp, on="Policy_Number", how='left')
te = pd.merge(te, tmp, on="Policy_Number", how='left')

In [138]:
tr.nequipment9_v2.value_counts()

0    210759
4         1
3         1
2         1
1         1
Name: nequipment9_v2, dtype: int64

In [141]:
tr["nequipment9"] = np.where(tr.nequipment9_v2 == 0, 0, 1 )
te["nequipment9"] = np.where(te.nequipment9_v2 == 0, 0, 1 )

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
claim["cause"] = le.fit_transform(claim.Cause_of_Loss)

In [ ]:
claim[claim.Policy_Number == "0004a3f30deef9fc8f9f5cb334b8bec8d79c7453"]

In [ ]:
tmp = claim.groupby("Policy_Number", as_index=0).agg({"Cause_of_Loss":"nunique"})
tmp.columns = ["Policy_Number", "nunique_Cause_of_Loss"]
tr = pd.merge(tr, tmp, on="Policy_Number", how='left')
te = pd.merge(te, tmp, on="Policy_Number", how='left')

In [ ]:
tmp = claim.groupby(["Policy_Number", "Cause_of_Loss"], as_index=0)["Claim_Number"].count()
tmp.Claim_Number = np.where(tmp.Claim_Number > 0, 1, 0)
tmp = tmp.pivot_table(columns="Cause_of_Loss", values="Claim_Number", index="Policy_Number")
tmp.columns = ["cause_of_claim_" + str(i) for i in range(17)]
tmp["Policy_Number"] = tmp.index
#tmp = tmp.fillna(0)

In [ ]:
tr = pd.merge(tr, tmp, on="Policy_Number", how='left')
te = pd.merge(te, tmp, on="Policy_Number", how='left')

In [ ]:
claim["Accident_month"] = claim.Accident_Date.str.split("/").str[1].astype('int')

In [ ]:
claim = claim.groupby(["Policy_Number", "Claim_Number"], as_index=0)["Accident_month"].first()
claim = claim.sort_values(["Policy_Number","Accident_month"],ascending=[1,1])

In [ ]:
claim["diff"] = claim.groupby('Policy_Number')["Accident_month"].diff()

In [ ]:
tmp = claim.groupby("Policy_Number", as_index=0).agg({"diff":['min','max','mean']})
tmp.columns = ["Policy_Number", "min_diff","max_diff","mean_diff"]

In [ ]:
tr = pd.merge(tr, tmp, on="Policy_Number", how='left')
te = pd.merge(te, tmp, on="Policy_Number", how='left')

In [ ]:
policy["dbirth_month"] = policy.dbirth.str.split("/").str[0]

In [ ]:
policy.dbirth_month = policy.dbirth_month.fillna(0)

In [ ]:
policy.dbirth_month = policy.dbirth_month.astype('int')

In [ ]:
policy["ibirth_month"] = policy.ibirth.str.split("/").str[0]
policy.ibirth_month = policy.ibirth_month.fillna(0)
policy.ibirth_month = policy.ibirth_month.astype('int')

In [ ]:
tmp = policy.groupby("Policy_Number", as_index=0)[["ibirth_month","dbirth_month"]].first()


In [ ]:
tr = pd.merge(tr, tmp, on="Policy_Number", how='left')
te = pd.merge(te, tmp, on="Policy_Number", how='left')

In [ ]:
tr['d_age_age_diff'] = tr.d_age - tr.age
te['d_age_age_diff'] = te.d_age - te.age

tr['d_age_age_ratio'] = tr.age - tr.d_age
te['d_age_age_ratio'] = te.age - te.d_age

In [73]:
# Insured's_ID
tmp = policy.groupby("Vehicle_identifier", as_index=0).agg({"Policy_Number":'nunique'})
tmp.columns = ["Vehicle_identifier", "Vehicle_identifier_nunique_policy"]
tmp2 = pd.merge(policy, tmp, on="Vehicle_identifier", how='left')
tmp2 = tmp2.groupby("Policy_Number",as_index=0)["Vehicle_identifier_nunique_policy"].first()
tr = pd.merge(tr, tmp2, on="Policy_Number", how='left')
te = pd.merge(te, tmp2, on="Policy_Number", how='left')

In [90]:
tmp = policy.groupby("Insured's_ID", as_index=0).agg({"Policy_Number":'nunique'})
tmp.columns = ["Insured's_ID", "Insured_ID_nunique_policy"]
policy = pd.merge(policy, tmp, on="Insured's_ID", how='left')
tmp = policy.groupby("Policy_Number", as_index=0)["Insured_ID_nunique_policy"].first()
tr = pd.merge(tr, tmp, on="Policy_Number", how='left')
te = pd.merge(te, tmp, on="Policy_Number", how='left')

In [96]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
policy["Insured_ID"] = le.fit_transform(policy["Insured's_ID"])

In [100]:
tmp = policy.groupby("Policy_Number",as_index=0)["Insured_ID"].first()
tr = pd.merge(tr, tmp, on="Policy_Number", how='left')
te = pd.merge(te, tmp, on="Policy_Number", how='left')

In [106]:
tmp = policy.groupby("Vehicle_identifier", as_index=0).agg({"Policy_Number":'nunique'})
tmp.columns = ["Vehicle_identifier", "Vehicle_identifier_nunique_policy"]

In [126]:
tmp = policy.groupby("Insured_ID", as_index=0).agg({"Vehicle_identifier":'nunique'})
tmp.columns = ["Insured_ID", 'Insured_ID_Vehicle_count']
tr = pd.merge(tr, tmp, on="Insured_ID", how='left')
te = pd.merge(te, tmp, on="Insured_ID", how='left')

In [26]:
tr.to_csv(path+'tr9.csv', encoding='utf-8', index=False)
te.to_csv(path+'te9.csv', encoding='utf-8', index=False)